In [45]:
import cv2
import numpy as np
import os
import shutil

In [46]:
EXTRACT_FOLDER = './img'  # 存放帧图片的位置
EXTRACT_FREQUENCY = 100  # 帧提取频率

In [47]:
video = cv2.VideoCapture("./video.mp4")
# 获取opencv的分类器——人眼识别和人脸识别
face_Cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [48]:
def extract_frames(dst_folder, index):
    frame_count = 0

    # 循环遍历视频中的所有帧
    while True:
        # 逐帧读取
        _, frame = video.read()
        if frame is None:
            break
        # 按照设置的频率保存图片
        if frame_count % EXTRACT_FREQUENCY == 0:
            # 设置保存文件名
            save_path = "{}/{:>03d}.jpg".format(dst_folder, index)
            # 保存图片
            cv2.imwrite(save_path, frame)
            index += 1  # 保存图片数＋1
        frame_count += 1  # 读取视频帧数＋1

    # 视频总帧数
    print(f'the number of frames: {frame_count}')
    # 打印出所提取图片的总数
    print("Totally save {:d} imgs".format(index - 1))

    # 计算FPS 方法一 get()
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')  # Find OpenCV version
    # (major_ver, minor_ver, subminor_ver) = (4, 5, 4)
    if int(major_ver) < 3:
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)  # 获取当前版本opencv的FPS
        print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
    else:
        fps = video.get(cv2.CAP_PROP_FPS)  # 获取当前版本opencv的FPS
        print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

    video.release()

In [49]:
try:
    shutil.rmtree(EXTRACT_FOLDER)
except OSError:
    pass
os.mkdir(EXTRACT_FOLDER)
# 抽取帧图片，并保存到指定路径
extract_frames(EXTRACT_FOLDER, 1)

the number of frames: 1039
Totally save 11 imgs
Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.00672913174947


In [99]:
# 下面开始检测人脸
# 注意这里要重新开一次 VideoCapture
video = cv2.VideoCapture("./video.mp4")
# get size and fps of video
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)
font = cv2.FONT_HERSHEY_SIMPLEX
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

30.00672913174947


In [100]:
# create VideoWriter for saving
outVideo = cv2.VideoWriter('./output_video.mp4', fourcc, fps, (width, height))
print((width, height))

(1920, 1080)


In [101]:
success, frame = video.read()
idx = 0

while success:
    # 将每帧图片灰度化
    size = frame.shape[:2]
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 直方图均衡
    image = cv2.equalizeHist(image)
    im_h, im_w = size
    minSize_1 = (im_w // 10, im_h // 10)
    # 获得人脸识别结果
    face_rect_results = face_Cascade.detectMultiScale(image, 1.05, 2, cv2.CASCADE_SCALE_IMAGE, minSize_1)
    if len(face_rect_results) > 0:
        # 绘制当前帧结果
        rect = []
        for res in face_rect_results:
            x, y, w, h = res
            rect.append([x, y, w, h])
            # 绘制人脸框
            cv2.rectangle(frame, (x, y), (x + w, y + h), [255, 255, 0], 2)

    # 输出每一帧结果
    print("idx=" + str(idx) + ", rect=" + str(rect))
    outVideo.write(frame)
    last_frame = frame
    idx = idx + 1
    success, frame = video.read()

video.release()
outVideo.release()

idx=0, rect=[[1074, 183, 465, 465]]
idx=1, rect=[[274, 160, 262, 262], [1087, 206, 437, 437]]
idx=2, rect=[[1083, 214, 446, 446]]
idx=3, rect=[[1084, 228, 450, 450]]
idx=4, rect=[[1092, 230, 456, 456]]
idx=5, rect=[[1098, 248, 444, 444]]
idx=6, rect=[[1084, 236, 477, 477]]
idx=7, rect=[[1092, 256, 462, 462]]
idx=8, rect=[[1096, 258, 452, 452], [259, 139, 288, 288]]
idx=9, rect=[[1092, 257, 455, 455]]
idx=10, rect=[[1093, 256, 450, 450]]
idx=11, rect=[[1096, 242, 451, 451]]
idx=12, rect=[[1084, 224, 454, 454]]
idx=13, rect=[[1092, 212, 442, 442]]
idx=14, rect=[[245, 159, 245, 245], [1093, 189, 434, 434]]
idx=15, rect=[[1090, 162, 441, 441]]
idx=16, rect=[[1085, 146, 433, 433]]
idx=17, rect=[[1085, 134, 424, 424]]
idx=18, rect=[[1086, 116, 424, 424], [199, 9, 571, 571]]
idx=19, rect=[[1090, 116, 416, 416], [207, 12, 572, 572]]
idx=20, rect=[[1086, 113, 418, 418], [187, 10, 590, 590]]
idx=21, rect=[[302, 243, 233, 233], [1083, 114, 410, 410], [216, 35, 581, 581]]
idx=22, rect=[[1079, 116,

In [83]:
video = cv2.VideoCapture("./output_video.mp4")
EXTRACT_DETECT_FOLDER = './detect_img'
extract_frames(EXTRACT_DETECT_FOLDER, 1)

the number of frames: 1039
Totally save 11 imgs
Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.007


In [102]:
# MeanShift 进行图像追踪
# get size and fps of video
raw_trace_video = cv2.VideoCapture("./output_video.mp4")
width = int(raw_trace_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(raw_trace_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = raw_trace_video.get(cv2.CAP_PROP_FPS)
print(fps)
font = cv2.FONT_HERSHEY_SIMPLEX
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

30.007


In [103]:
trace_video = cv2.VideoWriter('./trace_output_video.mp4', fourcc, fps, (width, height))

In [105]:
success, frame = raw_trace_video.read()
mujian_pos = [199, 9, 571, 571]
zhengyi_pos = [1083, 114, 410, 410]
mujian_trace_window = (199, 9, 571, 571)
zhengyi_trace_window = (1083, 114, 410, 410)

mujian_roi = frame[mujian_pos[1]:mujian_pos[1] + mujian_pos[2], mujian_pos[0]:mujian_pos[0] + mujian_pos[3]]
zhengyi_roi = frame[zhengyi_pos[1]:zhengyi_pos[1] + zhengyi_pos[2], zhengyi_pos[0]:zhengyi_pos[0] + zhengyi_pos[3]]

# 计算直方图
# 转换色彩空间
mujian_hsv_roi = cv2.cvtColor(mujian_roi, cv2.COLOR_BGR2HSV)
zhengyi_hsv_roi = cv2.cvtColor(zhengyi_roi, cv2.COLOR_BGR2HSV)
# 计算直方图
mujian_roi_hist = cv2.calcHist([mujian_hsv_roi], [0], None, [180], [0, 180])
zhengyi_roi_hist = cv2.calcHist([zhengyi_hsv_roi], [0], None, [180], [0, 180])
# 归一化
cv2.normalize(mujian_roi_hist, mujian_roi_hist, 0, 255, cv2.NORM_MINMAX)
cv2.normalize(zhengyi_roi_hist, zhengyi_roi_hist, 0, 255, cv2.NORM_MINMAX)

# 目标追踪
# 设置窗口搜索终止条件：最大迭代次数，窗口中心漂移最小值
term_crit = (cv2.TermCriteria_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# 159 帧出去，291 帧回来
# 680 帧出去，760 帧回来
trace_idx = 0
while success:
    if trace_idx < 159:
        # 计算直方图的反向投影
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mujian_dst = cv2.calcBackProject([hsv], [0], mujian_roi_hist, [0, 180], 1)
        zhengyi_dst = cv2.calcBackProject([hsv], [0], zhengyi_roi_hist, [0, 180], 1)

        # 进行meanshift追踪
        ret1, mujian_trace_window = cv2.meanShift(mujian_dst, mujian_trace_window, term_crit)
        ret2, zhengyi_trace_window = cv2.meanShift(zhengyi_dst, zhengyi_trace_window, term_crit)

        # 将追踪的位置绘制在视频上，并进行显示
        if ret1:
            x, y, w, h = mujian_trace_window
            cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        if ret2:
            x2, y2, w2, h2 = zhengyi_trace_window
            cv2.rectangle(frame, (x2, y2), (x2 + w2, y2 + h2), 255, 2)
    elif 159 <= trace_idx < 291:
            # 计算直方图的反向投影
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mujian_dst = cv2.calcBackProject([hsv], [0], mujian_roi_hist, [0, 180], 1)

        # 进行meanshift追踪
        ret1, mujian_trace_window = cv2.meanShift(mujian_dst, mujian_trace_window, term_crit)

        # 将追踪的位置绘制在视频上，并进行显示
        if ret1:
            x, y, w, h = mujian_trace_window
            cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
    else:
        # 重新初始化
        zhengyi_pos = [1673, 105, 240, 240]
        zhengyi_trace_window = (1673, 105, 240, 240)
        zhengyi_roi = frame[zhengyi_pos[1]:zhengyi_pos[1] + zhengyi_pos[2], zhengyi_pos[0]:zhengyi_pos[0] + zhengyi_pos[3]]
        zhengyi_hsv_roi = cv2.cvtColor(zhengyi_roi, cv2.COLOR_BGR2HSV)
        zhengyi_roi_hist = cv2.calcHist([zhengyi_hsv_roi], [0], None, [180], [0, 180])
        cv2.normalize(zhengyi_roi_hist, zhengyi_roi_hist, 0, 255, cv2.NORM_MINMAX)

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mujian_dst = cv2.calcBackProject([hsv], [0], mujian_roi_hist, [0, 180], 1)
        zhengyi_dst = cv2.calcBackProject([hsv], [0], zhengyi_roi_hist, [0, 180], 1)

        # 进行meanshift追踪
        ret1, mujian_trace_window = cv2.meanShift(mujian_dst, mujian_trace_window, term_crit)
        ret2, zhengyi_trace_window = cv2.meanShift(zhengyi_dst, zhengyi_trace_window, term_crit)

        # 将追踪的位置绘制在视频上，并进行显示
        if ret1:
            x, y, w, h = mujian_trace_window
            cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        if ret2:
            x2, y2, w2, h2 = zhengyi_trace_window
            cv2.rectangle(frame, (x2, y2), (x2 + w2, y2 + h2), 255, 2)

    trace_idx = trace_idx + 1
    trace_video.write(frame)
    success, frame = raw_trace_video.read()

raw_trace_video.release()
trace_video.release()

In [106]:
video = cv2.VideoCapture("./trace_output_video.mp4")
EXTRACT_TRACE_FOLDER = './trace_img'
extract_frames(EXTRACT_TRACE_FOLDER, 1)

the number of frames: 1039
Totally save 11 imgs
Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.007
